In [13]:
import os, json
import jsonlines

DATA_DIR = "../data"
data_list = [
    "KoAlpaca/KoAlpaca_v1.1.jsonl",
    "KoChatGPT/kochatgpt_1_SFT.jsonl",
    "korquad-chat-v1/korquad-chat.json",
    "OIG-small-chip2-ko/oig-smallchip2-dedu.jsonl",
    "ShareGPT_DeepL/ko_alpaca_style_dataset.json",
    "kullm_v2/kullm-v2.jsonl",
    "KoAlpaca/ko_alpaca_data.json"]

### KoAlpaca v1.0 dataset

In [44]:
data_path = data_list[-1]

with open(os.path.join(DATA_DIR, data_path), "r") as f:
    koalpaca_v1_0 = json.load(f)

len(koalpaca_v1_0)

49620

### KoAlpaca v1.1 dataset

In [14]:
data_path = data_list[0]

koalpaca_v1_1 = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        koalpaca_v1_1.append(line)

len(koalpaca_v1_1)

21155

### KochatGPT SFT dataset

In [15]:
data_path = data_list[1]
with open(os.path.join(DATA_DIR, data_path), "r") as f:
    kochatgpt = json.load(f)

len(kochatgpt)

12000

### Korquad dataset

In [16]:
data_path = data_list[2]

korquad = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        korquad.append(line)

len(korquad)

9619

### OIG dataset

In [17]:
data_path = data_list[3]

oig = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        oig.append(line)

len(oig)

210282

### KULLM-v2 dataset

In [18]:
data_path = data_list[5]

kullm = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        kullm.append(line)

len(kullm)

152630

In [19]:
# print(koalpaca_v1_0[0])
print(koalpaca_v1_1[0])
print(kochatgpt[0])
print(korquad[0])
print(oig[0])
# print(sharegpt[0])
print(kullm[-11])

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}
{'prompt': '불고기용 고기 한우에요?', 'completion': '저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.', 'tokens': 193}
{'source': 'korquad-chat', 'text': '<sys>1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 

In [20]:
import re
from tqdm import tqdm

def to_dialog(data, inst_key, out_key, preprossing=True):
    dialog = []
    for sample in tqdm(data, total=len(data)):
        if preprossing:
            inst = re.sub(r"\?(\w)", r"?\n\1", sample[inst_key])
            output = re.sub(r"[.](\d+)[.]", r".\r\n\1.", sample[out_key])
        
        dialog.append(
            [{"from": "human", "value": inst},
             {"from": "bot", "value": output}]
        )
    
    return dialog
        

print("Unifying except for Korquad dataset")
unified_data = []

# koalpacav1.1
unified_data.extend(to_dialog(koalpaca_v1_1, "instruction", "output", "general"))
# kochatgpt
unified_data.extend(to_dialog(kochatgpt, "prompt", "completion", "general"))
# OIG
unified_data.extend(to_dialog(oig, "user_translated", "chip2_translated", "general"))
    
# KULLM-v2
processed_kullm = []
for sample in tqdm(kullm, total=len(kullm)):
    if sample["input"]:
        inst = sample["instruction"].strip()+"\n\n"+sample["input"].strip()
    else:
        inst = sample["instruction"].strip()
        
    if len(inst) <= 15:
        continue
    
    output = re.sub(r"[.](\d+)[.]", r".\r\n\1.", sample["output"])
        
    processed_kullm.append(
        [{"from": "human", "value": inst},
         {"from": "bot", "value": output}]
    )
print(len(processed_kullm))
unified_data.extend(processed_kullm)

# unified_data.extend(koalpaca_v1_0)
len(unified_data)

Unifying except for Korquad dataset


100%|██████████| 152630/152630 [00:00<00:00, 199452.41it/s]

137583


381020

### Processing Korquard dataset

In [21]:
for sample in korquad:#, total=len(korquad)):
    try:
        turns = sample["text"].split("\n")
        context = turns[0].split(">")[1].strip()
        dialog = []
        for turn in turns[1:]:
            splited = turn.split(">")
            role = splited[0][1:]
            text = splited[1].strip()
            dialog.append({"role": role, "text": text})
            
        assert dialog[0]["role"]=="usr" and len(dialog)%2==0
        
        inst_res = []
        for i in range(0, len(dialog), 2):
            user = dialog[i]["text"]
            if i==0:
                user = context.strip()+"\n\n"+user.strip()
            bot = dialog[i+1]["text"]
        
            inst_res.extend(
                [{"from": "human", "value": user},
                 {"from": "bot", "value": bot}]
            )
        
        unified_data.append(inst_res)
    except:
        pass
    
len(unified_data)

390248

### AI Hub Book summarization

In [22]:
import os
from glob import glob
from tqdm import tqdm

json_files = glob("../data/AIhub_book_summary/*.json")

ins1_format = "다음 문단은 \'{kdc_label}\'로 분류된 문서의 일부분이다.\n\n{passage1}\n\n이 문단에 이어질 한 문장을 작성해줘"
res1_format = "\"{passage2}\"가 될 수 있겠습니다."
ins2_format = "완성된 문단을 간단히 요약하고 해당 문서의 제목을 지어줘"
res2_format = "이 문단은 다음과 같이 요약될 수 있습니다.\r\n- 요약: {summary}\r\n\r\n- {kdc_label} {doc_type} 제목 제안: \"{doc_name}\""

target_kdc = ["보건의료", "체육", "보육·가족및여성", "한국문학", "한국어", "문학", "문화재", "역사", "음악"]
book_summary = []
kdcs = []
for path in tqdm(json_files):
    with open(path, "r") as f:
        json_data = json.load(f)
    data = json_data["metadata"]
    kdcs.append(data["kdc_label"])
        
    if "교육" in data["kdc_label"] or data["kdc_label"] in target_kdc:
        splited_passage = json_data["passage"].split(". ")
        data["passage1"] = ". ".join(splited_passage[:-1])+"."
        data["passage2"] = splited_passage[-1]
        data["summary"] = json_data["summary"]
        ins1 = ins1_format.format_map(data)
        res1 = res1_format.format_map(data)
        ins2 = ins2_format.format_map(data)
        res2 = res2_format.format_map(data)
        book_summary.append([
            {"from":"human", "value":ins1},
            {"from":"bot", "value":res1},
            {"from":"human", "value":ins2},
            {"from":"bot", "value":res2},
        ])
    #     data["passage"] = json_data["passage"]
    #     data["summary"] = json_data["summary"]
    #     ins = ins_format.format_map(data)
    #     res = res_format.format_map(data)
    #     kdcs.append(data["kdc_label"])
    #     book_summary.append({"instruction":ins, "output":res})

100%|██████████| 180001/180001 [00:05<00:00, 30245.99it/s]


In [23]:
unified_data.extend(book_summary)
len(unified_data)

422474

### AI hub 주제별 일상 대화 데이터

In [24]:
from glob import glob
import json

chitchat_list = glob("../data/AIhub_chitchat/*.json")
chitchat_data = []

for path in chitchat_list:
    try:
        with open(path, "r") as fin:
            chitchat_data.append(json.load(fin))
    except Exception:
        print(path)
        continue

len(chitchat_data)

../data/AIhub_chitchat/KAKAO_1648_13.json


98651

In [25]:
from tqdm import tqdm
import re

dialogs = []
for data in chitchat_data:
    data = data["info"][0]
    domain = data["annotations"]["speaker_type"]
    if data["annotations"]["speaker_type"]!="1:1":
        continue
    
    data = data["annotations"]
    lines = data["lines"]
    
    if len(lines) < 2 or not data["lines"][0]["norm_text"]:
        continue
    
    if len(set([line["speaker"]["id"] for line in lines]))!=2:
        continue
       
    user_info = lines[0]["speaker"]
    u_id = user_info["id"]
    u_gender = user_info["sex"]
    u_age = user_info["age"]
    
    prev_id = u_id
    dialog = []
    user = []
    bot = []
    for i in range(len(lines)):
        cur_id = lines[i]["speaker"]["id"]
        if prev_id!=cur_id and prev_id!=u_id:
            response = " ".join(bot).strip()
            response = re.sub("키키", "ㅋㅋ", response)
            dialog.extend(
                [{"from": "human", "value": " ".join(user).strip()},
                 {"persona": {"age": a_age, "gender": a_gender, "domain": "일상 대화"}, "from": "bot", "value": response}]
            )
            user = []
            bot = []

        if cur_id!=u_id:
            a_gender = lines[i]["speaker"]["sex"]
            a_age = lines[i]["speaker"]["age"]
            bot.append(lines[i]["norm_text"])
        else:
            user.append(lines[i]["norm_text"])
            
        prev_id = cur_id
    
    if dialog:
        dialogs.append(dialog.copy())
    
print(len(dialogs))

unified_data.extend(dialogs)

70613


In [26]:
len(unified_data)

493087

In [12]:
from tqdm import tqdm
import torch
import transformers
from transformers import (
    MODEL_FOR_CAUSAL_LM_MAPPING,
    Trainer,
    set_seed,
    AutoConfig,
    CONFIG_MAPPING
)

import sys
sys.path.append("../")

from model_utils.conversation import get_conv_template, SeparatorStyle
from typing import Optional, Dict, Sequence

IGNORE_INDEX = -100

def preprocess(
    sources,
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    conv_g = get_conv_template("elementgpt_for_general")
    conv_p = get_conv_template("elementgpt_for_persona")
    roles = {"human": conv_g.roles[0], "bot": conv_g.roles[1]}

    # Apply prompt templates
    conversations = []
    for i, source in enumerate(sources):
        print(source)
        if "persona" in source[1]:
            conv = conv_p.copy()
            conv.system = conv.system.format_map(source[1]["persona"])
        else:
            conv = conv_g
            
        if roles[source[0]["from"]] != conv.roles[0]:
            # Skip the first one if it is not from human
            source = source[1:]

        conv.messages = []
        for j, sentence in enumerate(source):
            role = roles[sentence["from"]]
            assert role == conv.roles[j % 2], f"{i}"
            conv.append_message(role, sentence["value"])
        conversations.append(conv.get_prompt())
        
    return conversations
    
    # Tokenize conversations
    input_ids = []
    targets = []
    for i, conversation in enumerate(tqdm(conversations, desc="Tokenizing inputs")):
        input_ids_sample = tokenizer(
            conversation,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        ).input_ids[0]
        
        input_ids.append(input_ids_sample)
        targets.append(input_ids_sample.clone())

    assert conv.sep_style == SeparatorStyle.ADD_COLON_TWO

    print("Masking targets...")
    # Mask targets
    sep = conv.sep + conv.roles[1] + ": "
    for conversation, target in tqdm(zip(conversations, targets), total=len(conversations)):
        total_len = int(target.ne(tokenizer.pad_token_id).sum()) + 1

        rounds = conversation.split(conv.sep2)
        cur_len = 1
        target[:cur_len] = IGNORE_INDEX
        for i, rou in enumerate(rounds):
            if rou == "":
                break

            parts = rou.split(sep)
            if len(parts) != 2:
                break
            parts[0] += sep
            round_len = len(tokenizer(rou).input_ids)
            instruction_len = len(tokenizer(parts[0]).input_ids) - 2

            target[cur_len : cur_len + instruction_len] = IGNORE_INDEX

            cur_len += round_len
        target[cur_len:] = IGNORE_INDEX

        if False:
            z = target.clone()
            z = torch.where(z == IGNORE_TOKEN_ID, tokenizer.unk_token_id, z)
            rank0_print(tokenizer.decode(z))

        if cur_len < tokenizer.model_max_length:
            if cur_len != total_len:
                target[:] = IGNORE_INDEX
                print(
                    f"WARNING: tokenization mismatch: {cur_len} vs. {total_len}."
                    f" (ignored)"
                )
                print(len(rounds), conversation)

    return dict(
        input_ids=input_ids,
        labels=targets,
    )


In [13]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "EleutherAI/polyglot-ko-5.8b",
    cache_dir=None,
    model_max_length=1024,
    padding_side="right",
    use_fast=False,
)

processed = preprocess(dialogs[15:16], tokenizer)

[{'from': 'human', 'value': '일본 대마도에서 절도범들이 훔쳐 들어온 금동불상 얘기 들었어?'}, {'persona': {'age': '30대', 'gender': '여성', 'domain': '일상 대화'}, 'from': 'bot', 'value': '일본 어디서 가져왔는데?'}, {'from': 'human', 'value': '가져온 게 아니고, 훔쳤는데... 관음사에서.'}, {'persona': {'age': '30대', 'gender': '여성', 'domain': '일상 대화'}, 'from': 'bot', 'value': '훔쳤다면 돌려줘야겠네.'}, {'from': 'human', 'value': '근데 그게 진품이라서... 부석사에서 불상에 대한 소유권을 주장하고...'}, {'persona': {'age': '30대', 'gender': '여성', 'domain': '일상 대화'}, 'from': 'bot', 'value': '우리나라에서는 그래도 일본에 돌려줘야할텐데'}, {'from': 'human', 'value': '그래서 부석사가 정부 상대로 민사소소을 제기했다네...'}, {'persona': {'age': '30대', 'gender': '여성', 'domain': '일상 대화'}, 'from': 'bot', 'value': '음... 일본이 훔쳐갔던거라면 다시 돌려받아야지'}, {'from': 'human', 'value': '정식으로 돌려받았다면 좋았을텐데... 이게 다시 훔친 거라서... ;'}, {'persona': {'age': '30대', 'gender': '여성', 'domain': '일상 대화'}, 'from': 'bot', 'value': '아무튼 잘 해결되었음 좋겠다.'}]


In [27]:
import json

with open("../data/unified_instruction_add_summary.json", "w", encoding="utf-8") as f:
    json.dump(unified_data, f, ensure_ascii=False) # ensure_ascii로 한글이 깨지지 않게 저장

In [28]:
len(unified_data)

493087